<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

In [4]:
response.text

'{"timestamp": 1620976192, "iss_position": {"longitude": "-68.3123", "latitude": "47.2457"}, "message": "success"}'

Print the status code of the response:

In [3]:
#ANSWER:
response.status_code
# response.headers['Content-Type']

200

We can also request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21). Create a dict named `parameters` whose elements are key:value pairs named `lat` and `lon`, containing those figures:

In [5]:
#ANSWER:
parameters = {"lat": -33.87, "lon": 151.21}

The API request we want starts like the last one, but the last part of the endpoint is "iss-pass.json", and it takes a second argument -- namely, the `parameters` value created above. Compose and execute the API request:

In [6]:
#ANSWER:
response = requests.get("http://api.open-notify.org/iss-pass.json", params = parameters)

Print the response header:

In [7]:
#ANSWER:
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Fri, 14 May 2021 07:12:02 GMT', 'Content-Type': 'application/json', 'Content-Length': '521', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}


Print the content of the response (the data that the server returned):

In [8]:
#ANSWER:
print(response.content)

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1620974662, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 377, \n      "risetime": 1620979388\n    }, \n    {\n      "duration": 621, \n      "risetime": 1621027722\n    }, \n    {\n      "duration": 634, \n      "risetime": 1621033520\n    }, \n    {\n      "duration": 453, \n      "risetime": 1621039477\n    }, \n    {\n      "duration": 350, \n      "risetime": 1621045441\n    }\n  ]\n}\n'


Note that this is a Python byte string:

In [10]:
# print the type
print(type(response.content))

<class 'bytes'>


In [12]:
response.text

'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1620974662, \n    "latitude": -33.87, \n    "longitude": 151.21, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 377, \n      "risetime": 1620979388\n    }, \n    {\n      "duration": 621, \n      "risetime": 1621027722\n    }, \n    {\n      "duration": 634, \n      "risetime": 1621033520\n    }, \n    {\n      "duration": 453, \n      "risetime": 1621039477\n    }, \n    {\n      "duration": 350, \n      "risetime": 1621045441\n    }\n  ]\n}\n'

Print just the "content-type" value from the header:

In [11]:
#ANSWER:
print(response.headers["content-type"])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [13]:
overheads = response.json()
print(overheads)

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1620974662, 'latitude': -33.87, 'longitude': 151.21, 'passes': 5}, 'response': [{'duration': 377, 'risetime': 1620979388}, {'duration': 621, 'risetime': 1621027722}, {'duration': 634, 'risetime': 1621033520}, {'duration': 453, 'risetime': 1621039477}, {'duration': 350, 'risetime': 1621045441}]}


What kind (type) of object did this give us?

In [14]:
#ANSWER:
print(type(overheads))

<class 'dict'>


Python dicts  {key : value} are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `response` value to a separate dict:

In [29]:
#ANSWER:
passes = overheads['response']

In [30]:
print(passes)

[{'duration': 377, 'risetime': 1620979388}, {'duration': 621, 'risetime': 1621027722}, {'duration': 634, 'risetime': 1621033520}, {'duration': 453, 'risetime': 1621039477}, {'duration': 350, 'risetime': 1621045441}]


In [16]:
passes[1]['risetime']

1621027722

In [17]:
x = []
for item in passes:
    x.append(item['risetime'])
x

[1620979388, 1621027722, 1621033520, 1621039477, 1621045441]

Now extract the `risetime` strings into an array called `srisetimes`:

In [19]:
#ALT. ANSWER:
srisetimes = [xpass['risetime'] for xpass in passes]
srisetimes

[1620979388, 1621027722, 1621033520, 1621039477, 1621045441]

These values are in Unix Timestamp format.  Convert these to an array of Python `datetime` values called `risetimes`:

```
datetime.fromtimestamp(str)
```

In [20]:
#ANSWER:
risetimes = [datetime.fromtimestamp(xpass['risetime']) for xpass in passes] 

In [21]:
risetimes

[datetime.datetime(2021, 5, 14, 16, 3, 8),
 datetime.datetime(2021, 5, 15, 5, 28, 42),
 datetime.datetime(2021, 5, 15, 7, 5, 20),
 datetime.datetime(2021, 5, 15, 8, 44, 37),
 datetime.datetime(2021, 5, 15, 10, 24, 1)]

Finally, print these in a format that people understand:

In [22]:
#ANSWER:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

14/05/21 04:03
15/05/21 05:28
15/05/21 07:05
15/05/21 08:44
15/05/21 10:24


Here is an endpoint that tells us who is onboard:

In [23]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [24]:
response.text

'{"number": 7, "message": "success", "people": [{"name": "Mark Vande Hei", "craft": "ISS"}, {"name": "Oleg Novitskiy", "craft": "ISS"}, {"name": "Pyotr Dubrov", "craft": "ISS"}, {"name": "Thomas Pesquet", "craft": "ISS"}, {"name": "Megan McArthur", "craft": "ISS"}, {"name": "Shane Kimbrough", "craft": "ISS"}, {"name": "Akihiko Hoshide", "craft": "ISS"}]}'

In [25]:
response.content

b'{"number": 7, "message": "success", "people": [{"name": "Mark Vande Hei", "craft": "ISS"}, {"name": "Oleg Novitskiy", "craft": "ISS"}, {"name": "Pyotr Dubrov", "craft": "ISS"}, {"name": "Thomas Pesquet", "craft": "ISS"}, {"name": "Megan McArthur", "craft": "ISS"}, {"name": "Shane Kimbrough", "craft": "ISS"}, {"name": "Akihiko Hoshide", "craft": "ISS"}]}'

In [26]:
#ANSWER:
astros = response.json()
print(astros)
print(astros["number"])
[print(astronaut['name']) for astronaut in astros['people']]

{'number': 7, 'message': 'success', 'people': [{'name': 'Mark Vande Hei', 'craft': 'ISS'}, {'name': 'Oleg Novitskiy', 'craft': 'ISS'}, {'name': 'Pyotr Dubrov', 'craft': 'ISS'}, {'name': 'Thomas Pesquet', 'craft': 'ISS'}, {'name': 'Megan McArthur', 'craft': 'ISS'}, {'name': 'Shane Kimbrough', 'craft': 'ISS'}, {'name': 'Akihiko Hoshide', 'craft': 'ISS'}]}
7
Mark Vande Hei
Oleg Novitskiy
Pyotr Dubrov
Thomas Pesquet
Megan McArthur
Shane Kimbrough
Akihiko Hoshide


[None, None, None, None, None, None, None]

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [27]:
#ANSWER:
def handleResponse(response, verbose = False):
    if response.status_code == 200:
        if verbose:
            print('Response = 200: OK')
        fail = False
    else:
        if verbose:
            print('Response = ' + str(response.status_code) + ': ' + '(TODO: output of lookup function here)')
        fail = True
    return(fail, response.status_code)

2. Test your response handler on some correct and incorrect API calls.

In [28]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-pass.json")
handleResponse(response, True)[0]

Response = 400: (TODO: output of lookup function here)


True

>
>
>
>
>



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



